In [43]:
import numpy as np
import pandas as pd
import utilities
import statistics
import matplotlib.pyplot as plt
import torch
from gcn_model import GCNModel
from sklearn.metrics import confusion_matrix

In [2]:
spg_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102982_Pouf5SPGmat.txt", index_col = 0)
spc1_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102983_SPCImat.txt", index_col = 0)
spc2_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102984_SPCIImat.txt", index_col = 0)
spd_data = pd.read_table("/home/groups/ConradLab/daniel/GNN_cell_annotation/data/jung/GSM3102985_SPDmat.txt", index_col = 0)

In [3]:
spc1_data.head()

,GTTTTCTCGTT,GGGACCACGCC,GTCGTGCATTT,GTTGACTTCCG,AATAACACAGC,GCTTCTTTGCT,GGTTTTCTCGT,ACAGTCAGGGT,TCCCTAATCAG,GTCGTCATTTA,...,TTTCTTCCCAT,CGATGGGTTTT,AACATACAAAG,CCTCTTTCATT,ATCCCTTTCGA,CCCGCGTCGCT,GGGTTCACGCC,GTCACTAGGTC,GTCCCAATGTC,GTTCATTACAC
0610007P14Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009B22Rik,1,1,0,2,0,1,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
0610009L18Rik,4,4,0,1,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
0610009O20Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610010F05Rik,0,2,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
merged_data = spg_data.merge(spc1_data, how='inner', left_index=True, right_index=True)
merged_data = merged_data.merge(spc2_data, how='inner', left_index=True, right_index=True)
merged_data = merged_data.merge(spd_data, how='inner', left_index=True, right_index=True)

In [5]:
merged_data.shape

(6354, 1192)

In [6]:
#merged_data.loc["Dmrt1",:]

In [7]:
merged_data.loc[["0610009L18Rik", "0610012G03Rik"], "TTCGTGTCGAA"]

0610009L18Rik    0
0610012G03Rik    2
Name: TTCGTGTCGAA, dtype: int64

In [8]:
genes = merged_data.index.array
cell_names = merged_data.columns.array

In [9]:
merged_data = np.asmatrix(merged_data)
merged_data = merged_data.transpose()
merged_data.shape

(1192, 6354)

In [10]:
spg_labels = ["spg"] * len(spg_data.columns)
spc1_labels = ["spc"] * len(spc1_data.columns)
spc2_labels = ["spc"] * len(spc2_data.columns)
spd_labels = ["spd"] * len(spd_data.columns)
merged_labels = spg_labels + spc1_labels + spc2_labels + spd_labels
len(merged_labels)

1192

In [11]:
unique, counts = np.unique(merged_labels, return_counts=True)
counts

array([597, 299, 296])

In [12]:
# read in dataset
X, keep_cells,keep_genes,pca_obj = utilities.preprocess(np.array(merged_data), normalize=False, run_pca=False, scale=False)
X.shape

(692, 6354)

In [13]:
merged_labels = np.array(merged_labels)[keep_cells]
cell_names = cell_names[keep_cells]
len(merged_labels)

692

In [14]:
unique, counts = np.unique(merged_labels, return_counts=True)
counts

array([244, 156, 292])

In [15]:
# randomize
np.random.seed(8)
p = np.random.permutation(len(merged_labels))
merged_labels = merged_labels[p]
X = X[p,:]
cell_names = cell_names[p]

In [16]:
# write out
final_counts = pd.DataFrame(X)
final_counts.index = cell_names
final_counts.columns = genes

In [17]:
final_counts.columns.isin(["Dmc1"]).any()

False

In [18]:
final_counts.head()

,0610009B22Rik,0610009L18Rik,0610009O20Rik,0610010F05Rik,0610011F06Rik,0610012G03Rik,0610030E20Rik,0610037L13Rik,0610040B10Rik,1110001J03Rik,...,Zzz3,mt-Co1,mt-Cytb,mt-Nd1,mt-Nd2,mt-Nd3,mt-Nd4,mt-Nd5,mt-Rnr1,mt-Rnr2
TTCGTGTCGAA,0,0,0,0,0,2,0,0,0,0,...,0,12,30,27,12,0,15,8,9,32
GTACAATGATG,0,0,0,1,0,2,0,0,1,0,...,1,49,58,39,18,1,18,18,27,88
TCGAGGACGTT,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
GTGAATTTTCT,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
CCACACGCGGT,0,0,0,0,0,0,0,0,0,0,...,1,24,29,29,7,0,16,6,11,50


In [19]:
final_counts.loc["TTCGTGTCGAA", ["0610009L18Rik", "0610012G03Rik"]]

0610009L18Rik    0
0610012G03Rik    2
Name: TTCGTGTCGAA, dtype: int64

In [20]:
merged_labels = pd.DataFrame(merged_labels)
merged_labels.index = cell_names
merged_labels

,0
TTCGTGTCGAA,spg
GTACAATGATG,spg
TCGAGGACGTT,spd
GTGAATTTTCT,spd
CCACACGCGGT,spg
...,...
ACGCATTCAAT,spg
TCTGGAATCAA,spg
GCCCCCTCTTT,spc
GTGCACATATA,spc


In [21]:
#final_counts.to_csv("/home/groups/ConradLab/daniel/sharp_data/jung/counts.csv")
#merged_labels.to_csv("/home/groups/ConradLab/daniel/sharp_data/jung/labels.csv")

In [22]:
all_labels = pd.read_csv("/home/groups/ConradLab/daniel/sharp_data/jung/preds.csv", index_col=0)

In [23]:
_,marker_names = utilities.read_marker_file("/home/groups/ConradLab/daniel/sharp_data/jung/markers.txt")
marker_names

['spg', 'spc', 'spd']

In [24]:
all_labels_factored = utilities.factorize_df(all_labels, marker_names)
encoded_labels = utilities.encode_predictions(all_labels_factored)
encoded_labels

array([[0., 0., 3.],
       [0., 0., 3.],
       [0., 3., 0.],
       ...,
       [2., 0., 0.],
       [3., 0., 0.],
       [0., 3., 0.]])

In [25]:
all_labels_factored

,scina,scsorter,sctype
0,2,2,2
1,2,2,2
2,1,1,1
3,1,1,1
4,2,2,2
...,...,...,...
687,2,2,2
688,2,2,2
689,-1,0,0
690,0,0,0


In [26]:
merged_labels = pd.factorize(merged_labels.iloc[:,0], sort=True)[0]
merged_labels

array([2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1,
       1, 2, 1, 1, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 2, 2, 2, 2, 1, 2, 1, 2, 1, 0,
       2, 1, 2, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0, 2, 0, 2, 2, 0, 1, 0, 0, 2,
       2, 1, 2, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 1, 2, 0, 0,
       0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 2, 2,
       2, 0, 2, 1, 2, 1, 0, 0, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 0, 0, 0, 2,
       2, 1, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2, 0, 1,
       2, 0, 0, 2, 1, 1, 2, 2, 0, 2, 2, 0, 2, 2, 1, 0, 0, 0, 2, 0, 0, 2,
       2, 0, 2, 0, 0, 2, 1, 2, 1, 1, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 1, 0,
       2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 2, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 1, 1, 0, 2,
       1, 2, 2, 0, 0, 0, 2, 1, 0, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1,
       0, 2, 1, 1, 0, 2, 1, 1, 0, 2, 1, 0, 2, 0, 2,

In [27]:
print(utilities.pred_accuracy(all_labels_factored['scina'], merged_labels))
print(utilities.pred_accuracy(all_labels_factored['sctype'], merged_labels))
print(utilities.pred_accuracy(all_labels_factored['scsorter'], merged_labels))
#print(utilities.pred_accuracy(all_labels_factored['singler'], real_y))
#print(utilities.pred_accuracy(all_labels_factored['scpred'], real_y))


0.5390173196792603
0.8482658863067627
0.7745664715766907


In [28]:
max_pred = torch.tensor(encoded_labels).max(dim=1)[1]
utilities.pred_accuracy(max_pred, merged_labels)

/home/users/lewinsda/scSHARP/utilities.py:187: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return float((torch.tensor(preds) == torch.tensor(real)).type(torch.FloatTensor).mean().numpy())


0.852601170539856

In [29]:
for i in range(encoded_labels.shape[0]):
    row = encoded_labels[i,:]
    if row.sum() == 0:
        print(row)
        print(i)
        print(row/row.sum())

[0. 0. 0.]
19
[nan nan nan]
[0. 0. 0.]
82
[nan nan nan]
[0. 0. 0.]
172
[nan nan nan]
[0. 0. 0.]
182
[nan nan nan]
[0. 0. 0.]
250
[nan nan nan]
[0. 0. 0.]
272
[nan nan nan]
[0. 0. 0.]
340
[nan nan nan]
[0. 0. 0.]
389
[nan nan nan]
[0. 0. 0.]
490
[nan nan nan]
[0. 0. 0.]
525
[nan nan nan]
[0. 0. 0.]
545
[nan nan nan]
[0. 0. 0.]
554
[nan nan nan]
[0. 0. 0.]
582
[nan nan nan]
[0. 0. 0.]
649
[nan nan nan]
[0. 0. 0.]
684
[nan nan nan]


<ipython-input-29-5047c4126b95>:6: RuntimeWarning: invalid value encountered in true_divide
  print(row/row.sum())


In [30]:
confident_labels = utilities.get_consensus_labels(encoded_labels, necessary_vote = 2)
confident_labels.shape

(692,)

In [31]:
confident_labels[582]

-1.0

In [32]:
train_nodes = np.where(confident_labels != -1)[0]
test_nodes = np.where(confident_labels == -1)[0]
len(test_nodes)

74

In [33]:
print(utilities.pred_accuracy(confident_labels[train_nodes], merged_labels[train_nodes]))

0.8689320683479309


In [34]:
# tool accuracy on test
print(utilities.pred_accuracy(np.array(all_labels_factored['scina'][test_nodes]), merged_labels[test_nodes]))
print(utilities.pred_accuracy(np.array(all_labels_factored['sctype'][test_nodes]), merged_labels[test_nodes]))
print(utilities.pred_accuracy(np.array(all_labels_factored['scsorter'][test_nodes]), merged_labels[test_nodes]))
#print(utilities.pred_accuracy(np.array(all_labels_factored['singler'][test_nodes]), real_y[test_nodes]))
#print(utilities.pred_accuracy(np.array(all_labels_factored['scpred'][test_nodes]), real_y[test_nodes]))
#max_pred = torch.tensor(encoded_labels).max(dim=1)[1]
print(utilities.pred_accuracy(max_pred[test_nodes], merged_labels[test_nodes]))

0.0
0.6891891956329346
0.013513513840734959
0.7162162065505981


In [35]:
all_labels_factored.iloc[test_nodes,:]

,scina,scsorter,sctype
19,-1,-1,-1
29,-1,-1,0
51,-1,0,1
53,-1,1,0
70,-1,-1,1
...,...,...,...
658,-1,1,-1
659,-1,2,0
672,-1,0,-1
676,-1,1,0


In [36]:
unique, counts = np.unique(merged_labels[test_nodes], return_counts=True)
counts

array([53,  8, 13])

In [37]:
X, keep_cells,keep_genes,pca_obj = utilities.preprocess(X, normalize=True, run_pca=True, scale=False)

In [38]:
dataset  = torch.utils.data.TensorDataset(torch.tensor(X), torch.tensor(confident_labels))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=True)

test_dataset  = torch.utils.data.TensorDataset(torch.tensor(X), torch.tensor(merged_labels))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=20, shuffle=False)

In [39]:
X.shape

(692, 500)

In [40]:
m = GCNModel("configs/2_25.txt", 2, 3)
m.train(dataloader, 150)

Loss in epoch 0 = 16.163952
Loss in epoch 10 = 0.024151
Loss in epoch 20 = 0.005103
Loss in epoch 30 = 0.002394
Loss in epoch 40 = 0.001285
Loss in epoch 50 = 0.000991
Loss in epoch 60 = 0.000556
Loss in epoch 70 = 0.000450
Loss in epoch 80 = 0.000270
Loss in epoch 90 = 0.000244
Loss in epoch 100 = 0.000193
Loss in epoch 110 = 0.000136
Loss in epoch 120 = 0.000101
Loss in epoch 130 = 0.000097
Loss in epoch 140 = 0.000069


In [41]:
m.validation_metrics(test_dataloader, train_nodes, test_nodes)

(0.8699421882629395,
 array([[164,  75,   5],
        [ 10, 146,   0],
        [  0,   0, 292]]),
 0.8689320683479309,
 array([[120,  70,   1],
        [ 10, 138,   0],
        [  0,   0, 279]]),
 0.8783783912658691,
 array([[44,  5,  4],
        [ 0,  8,  0],
        [ 0,  0, 13]]))

In [47]:
confusion_matrix( merged_labels[test_nodes], max_pred[test_nodes])

array([[47,  5,  1],
       [ 4,  4,  0],
       [11,  0,  2]])